In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
# https://www.youtube.com/watch?v=jtIMnmbnOFo

query = "(from:elonmusk) until:2023-03-03 since:2023-01-01"
tweets = []
limit = 50

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([
            # tweet.date,
            # tweet.user.username,
            tweet.rawContent,
        ])

# df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
df = pd.DataFrame(tweets, columns=['Tweet'])

# replace "[^a-zA-Z0-9]", " "
df['Tweet'] = df['Tweet'].str.replace("[^a-zA-Z0-9]", " ")
df.to_csv('elonmusk_tweets.csv', index=False)

In [67]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

keyword = "bbm"
num_of_tweets = 20

# Buat list kosong untuk menyimpan hasil scrape
tweets_list = []

# Scrape data
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' lang:id').get_items()):
    if i>num_of_tweets:
        break
    tweets_list.append([
        tweet.date,
        tweet.id,
        tweet.user.username,
        tweet.likeCount,
        tweet.retweetCount,
        tweet.rawContent,
    ])

# Buat dataframe
tweets_df = pd.DataFrame(tweets_list, columns=[
    'Datetime', 'Tweet Id', 'Username', 'Likes', 'Retweets', 'Text'
])

# Tampilkan 5 data teratas
tweets_df.head()

# save
tweets_df.to_csv('bbm_tweets.csv', index=False)

In [33]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\sammi-
[nltk_data]     vian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\sammi-
[nltk_data]     vian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools
import pandas as pd
import pickle
from IPython.display import display, clear_output

In [37]:
rawdf = pd.read_csv('bbm_tweets.csv')
rawdf.head()

# get tweet text
tweet_text = rawdf['Text']
tweet_text

0     Mau dapet komisi, buat tambahan uang jajan, bu...
1     dongcaste Mau tampil keren dgn style anime kam...
2     Website Profesional dengan Harga Terjangkau di...
3     Pernah kena jahil temen dulu pas SMP dia pasan...
4     @horatio9797 @ZULFSW @misterjunker @punyae_cin...
5     @Pencari_Rezeki @amandasah__ @KPK_RI @DivHumas...
6     Pengamat K3: Buffer Zone terminal BBM lindungi...
7                                      @draaken bbm pak
8     Sobat Jatim, sudah tahu belum apa aja sih jeni...
9     @thedobbybackup2 @ShivSquad1 Fkt voting mhnje ...
10    Dirut Pertamina Ungkap Hasil Investigasi Kebak...
11    @kozirama Mungkin krn frekwensi sdh tinggi, di...
12    @winnershiv1 @ShivSquad1 Ha barobar bhava kon ...
13    @ReDFloW9002 @DimasSnjy78 @LivingCh @Heraloebs...
14    tapi, sekolah online bikin gua ngehargain mome...
15    Jasa iklan bbm ke 50.000 kontak bb bergaransi ...
16    @Thalomoan1 Apa aja di demo...😄😄\nMulai dr har...
17    Bos Pertamina: Simpan 15% Stok BBM Nasiona

In [38]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas

tweet_text = tweet_text.str.lower()
print('Case Folding Result : \n')
print(tweet_text.head(5))
print('\n\n\n')

Case Folding Result : 

0    mau dapet komisi, buat tambahan uang jajan, bu...
1    dongcaste mau tampil keren dgn style anime kam...
2    website profesional dengan harga terjangkau di...
3    pernah kena jahil temen dulu pas smp dia pasan...
4    @horatio9797 @zulfsw @misterjunker @punyae_cin...
Name: Text, dtype: object






In [29]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [39]:
# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

tweet_text = tweet_text.apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

tweet_text = tweet_text.apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

tweet_text = tweet_text.apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

tweet_text = tweet_text.apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

tweet_text = tweet_text.apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

tweet_text = tweet_text.apply(remove_singl_char)

# NLTK word rokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

rawdf['token'] = tweet_text.apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(rawdf['token'].head(5))
print('\n\n\n')

Tokenizing Result : 

0    [mau, dapet, komisi, buat, tambahan, uang, jaj...
1    [dongcaste, mau, tampil, keren, dgn, style, an...
2    [website, profesional, dengan, harga, terjangk...
3    [pernah, kena, jahil, temen, dulu, pas, smp, d...
4              [cinta, dibakar, juga, butuh, bbm, mas]
Name: token, dtype: object






In [68]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

rawdf['tweet_tokens_WSW'] = rawdf['token'].apply(stopwords_removal)

print(rawdf['tweet_tokens_WSW'].head(5))

0    [dapet, komisi, tambahan, uang, jajan, buruan,...
1    [dongcaste, tampil, keren, style, anime, follo...
2    [website, profesional, harga, terjangkau, tung...
3    [kena, jahil, temen, pas, smp, pasang, status,...
4                    [cinta, dibakar, butuh, bbm, mas]
Name: tweet_tokens_WSW, dtype: object


In [69]:
### --- Stemming --- ####

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in rawdf['tweet_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

rawdf['tweet_tokens_stemmed'] = rawdf['tweet_tokens_WSW'].swifter.apply(get_stemmed_term)
print(rawdf['tweet_tokens_stemmed'])

250
------------------------
dapet : dapet
komisi : komisi
tambahan : tambah
uang : uang
jajan : jajan
buruan : buru
bro : bro
daftar : daftar
reseller : reseller
menguntungkan : untung
info : info
bbm : bbm
bda : bda
dongcaste : dongcaste
tampil : tampil
keren : keren
style : style
anime : anime
follow : follow
distroartonline : distroartonline
sms : sms
bbmbaf : bbmbaf
website : website
profesional : profesional
harga : harga
terjangkau : jangkau
tunggu : tunggu
minat : minat
hub : hub
fbabb : fbabb
kena : kena
jahil : jahil
temen : temen
pas : pas
smp : smp
pasang : pasang
status : status
nama : nama
ni : ni
cowok : cowok
pikir : pikir
pacar : pacar
barunya : baru
cengin : cengin
dianya : dia
senyum : senyum
gitu : gitu
percaya : percaya
ehhh : ehhh
taunya : tau
cinta : cinta
dibakar : bakar
butuh : butuh
mas : mas
rezeki : rezeki
ri : ri
polri : polri
prestasi : prestasi
ahok : ahok
pertamina : pertamina
untung : untung
depo : depo
terbakar : bakar
muraaahhh : muraaahhh
banget : ba

Pandas Apply:   0%|          | 0/21 [00:00<?, ?it/s]

0     [dapet, komisi, tambah, uang, jajan, buru, bro...
1     [dongcaste, tampil, keren, style, anime, follo...
2     [website, profesional, harga, jangkau, tunggu,...
3     [kena, jahil, temen, pas, smp, pasang, status,...
4                       [cinta, bakar, butuh, bbm, mas]
5     [rezeki, ri, polri, prestasi, ahok, pertamina,...
6     [amat, buffer, zone, terminal, bbm, lindung, m...
7                                                 [bbm]
8     [sobat, jatim, jenisjenis, bbm, tata, kini, da...
9     [fkt, voting, mhnje, support, nasto, khara, su...
10    [dirut, pertamina, hasil, investigasi, bakar, ...
11                  [frekwensi, pakai, ka, barang, bbm]
12    [ha, barobar, bhava, kon, tar, ahe, samjhun, g...
13    [abang, warga, tangerang, berita, jni, mobil, ...
14    [sekolah, online, gua, ngehargain, momen, doku...
15    [jasa, iklan, bbm, kontak, bb, garansi, rb, ja...
16    [demo, dr, harga, gas, harga, bbm, vaksin, cip...
17    [bos, pertamina, simpan, stok, bbm, nasion

In [45]:
rawdf.to_csv("preprocessing_bbm.csv")

In [44]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

for i in tweet_text:
  # stemming process
  sentence = i
  output   = stemmer.stem(sentence)

  print(output)
  # ekonomi indonesia sedang dalam tumbuh yang bangga


mau dapet komisi buat tambah uang jajan buru bro daftar jadi reseller kami untung info bbm bda
dongcaste mau tampil keren dgn style anime kamu follow distroartonline sms bbmbaf
website profesional dengan harga jangkau di mulai dari tunggu apa lagi minat hub bbm di fbabb
pernah kena jahil temen dulu pas smp dia pasang status nama ni cowok di bbm aku pikir itu pacar baru kan jadi aku cengin dia cuma senyum gitu jadi aku percaya ehhh tau
cinta bakar juga butuh bbm mas
rezeki ri polri prestasi ahok pertamina untung besar tidak pernah yg nama nya depo bakar harga bbm muraaahhh banget jargontaik konon kata
amat buffer zone terminal bbm lindung masyarakat dari bahaya beritaja
bbm pak
sobat jatim sudah tahu belum apa aja sih jenisjenis bbm atau tata cara kini daftar subsidi di my pertamina yuk simak lengkap di infografik ikut
fkt voting mhnje support nasto khara support tya nanatr chalu hoto he kalat nahi yana mhnaun gayab hotata same bbm nantr zala ani tyacha hype gela aata pn gayab aahet ani

In [46]:
import pandas as pd
import numpy as np

TWEET_DATA = pd.read_csv("preprocessing_bbm.csv", usecols=["tweet_tokens_stemmed"])
TWEET_DATA.columns = ["tweet"]

TWEET_DATA.head()

,tweet
0,"['dapet', 'komisi', 'tambah', 'uang', 'jajan',..."
1,"['dongcaste', 'tampil', 'keren', 'style', 'ani..."
2,"['website', 'profesional', 'harga', 'jangkau',..."
3,"['kena', 'jahil', 'temen', 'pas', 'smp', 'pasa..."
4,"['cinta', 'bakar', 'butuh', 'bbm', 'mas']"


In [49]:
# convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

TWEET_DATA["tweet_list"] = TWEET_DATA["tweet"].apply(convert_text_list)
TWEET_DATA.head()


,tweet,tweet_list
0,"['dapet', 'komisi', 'tambah', 'uang', 'jajan',...","[dapet, komisi, tambah, uang, jajan, buru, bro..."
1,"['dongcaste', 'tampil', 'keren', 'style', 'ani...","[dongcaste, tampil, keren, style, anime, follo..."
2,"['website', 'profesional', 'harga', 'jangkau',...","[website, profesional, harga, jangkau, tunggu,..."
3,"['kena', 'jahil', 'temen', 'pas', 'smp', 'pasa...","[kena, jahil, temen, pas, smp, pasang, status,..."
4,"['cinta', 'bakar', 'butuh', 'bbm', 'mas']","[cinta, bakar, butuh, bbm, mas]"


In [50]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        # rumus = frekuensi kemunculan term / jumlah term pada dokumen
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

TWEET_DATA["TF_dict"] = TWEET_DATA['tweet_list'].apply(calc_TF)
TWEET_DATA["TF_dict"].head()

0    {'dapet': 0.07692307692307693, 'komisi': 0.076...
1    {'dongcaste': 0.1111111111111111, 'tampil': 0....
2    {'website': 0.1111111111111111, 'profesional':...
3    {'kena': 0.047619047619047616, 'jahil': 0.0476...
4    {'cinta': 0.2, 'bakar': 0.2, 'butuh': 0.2, 'bb...
Name: TF_dict, dtype: object

In [55]:
index = 5

print('%20s' % "term", "\t", "TF\n")
for key in TWEET_DATA["TF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key])

                term 	 TF

              rezeki 	 0.0625
                  ri 	 0.0625
               polri 	 0.0625
            prestasi 	 0.0625
                ahok 	 0.0625
           pertamina 	 0.0625
              untung 	 0.0625
                nama 	 0.0625
                depo 	 0.0625
               bakar 	 0.0625
               harga 	 0.0625
                 bbm 	 0.0625
           muraaahhh 	 0.0625
              banget 	 0.0625
          jargontaik 	 0.0625
               konon 	 0.0625


In [56]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(TWEET_DATA["TF_dict"])

In [57]:
n_document = len(TWEET_DATA)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict

#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [58]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
TWEET_DATA["TF-IDF_dict"] = TWEET_DATA["TF_dict"].apply(calc_TF_IDF)

In [60]:
# Check TF-IDF result
index = 10

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in TWEET_DATA["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key] ,"\t" , TWEET_DATA["TF-IDF_dict"][index][key])


                term 	         TF 	              TF-IDF

               dirut 	 0.1111111111111111 	 0.26126391746260863
           pertamina 	 0.1111111111111111 	 0.15945383614325806
               hasil 	 0.1111111111111111 	 0.26126391746260863
         investigasi 	 0.1111111111111111 	 0.26126391746260863
               bakar 	 0.1111111111111111 	 0.18424756406705914
                depo 	 0.1111111111111111 	 0.18424756406705914
                 bbm 	 0.1111111111111111 	 0.01112038428410918
            plumpang 	 0.1111111111111111 	 0.21621223878392368
              tangki 	 0.1111111111111111 	 0.26126391746260863


In [61]:
# sort descending by value for DF dictionary
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

TWEET_DATA["TF_IDF_Vec"] = TWEET_DATA["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(TWEET_DATA["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(TWEET_DATA["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.007698727581306356, 0.0, 0.0, 0.0, 0.0, 0.14968539608117795, 0.14968539608117795, 0.14968539608117795, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.18087501978180598, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [63]:
# Convert Series to List
TF_IDF_Vec_List = np.array(TWEET_DATA["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))

ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
2,harga,0.656382
3,bakar,0.619532
1,pertamina,0.495161
26,pakai,0.491598
4,depo,0.453710
17,subsidi,0.446243
11,anime,0.432424
15,bbmbaf,0.432424
14,sms,0.432424
12,follow,0.432424


In [64]:
ranking.to_csv("bbm_ranking.csv")